In [78]:
from qiskit import QuantumCircuit, transpile
import random
from qiskit_aer import AerSimulator
simulator=AerSimulator()

In [79]:
no_qubits=int(input("Please Enter the no of qubits ="))

Please Enter the no of qubits = 200


In [80]:
#Step 2: Random Bit and Basis Generation , Alice and Bob each pick random bits and bases.
def generate_random_bits(n):
    return [random.randint(0, 1) for _ in range(n)]

def generate_random_bases(n):
    # 0 = rectilinear (Z), 1 = diagonal (X)
    return [random.randint(0, 1) for _ in range(n)]

In [81]:
#Step 3: Alice Encodes the Bits into Qubits
def create_bb84_qubits(bits, bases):
    circuits = []
    for bit, basis in zip(bits, bases):
        qc = QuantumCircuit(1, 1)
        if bit == 1:
            qc.x(0)  # flip qubit for bit=1
        if basis == 1:
            qc.h(0)  # switch to diagonal basis
        circuits.append(qc)
    return circuits

In [82]:
#Step 4: Bob Measures the Qubits
#Bob uses his random bases to measure.
#If bases match → correct bit;
#If not → random result.
def measure_qubits(circuits, bob_bases):
    qc=circuits
    compiled_circuit = transpile(qc, simulator)
    # Run the circuit on the simulator
    bob_results = []
    for qc, basis in zip(circuits, bob_bases):
        q = qc.copy()
        if basis == 1:
            q.h(0)  # measure in X basis
        q.measure(0, 0)
        job = simulator.run(q, shots=1)
        result = job.result()
        measured_bit = int(list(result.get_counts().keys())[0])
        bob_results.append(measured_bit)
    return bob_results

In [83]:
#Step 5: Sifting the Key
#Alice and Bob publicly compare bases, keep bits measured in matching bases.
def sift_key(alice_bits, alice_bases, bob_bits, bob_bases):
    sifted_alice, sifted_bob = [], []
    for a_bit, a_basis, b_bit, b_basis in zip(alice_bits, alice_bases, bob_bits, bob_bases):
        if a_basis == b_basis:
            sifted_alice.append(a_bit)
            sifted_bob.append(b_bit)
    return sifted_alice, sifted_bob


In [84]:
#Step 6: Eavesdrop Detection
#They reveal a random sample (e.g., 20%) to estimate the error rate.
def check_for_eavesdropping(alice_sifted, bob_sifted, sample_fraction=0.2):
    n = len(alice_sifted)
    if n == 0:
        return 1.0, [], []

    sample_size = max(1, int(n * sample_fraction))
    sample_indices = random.sample(range(n), sample_size)

    alice_sample = [alice_sifted[i] for i in sample_indices]
    bob_sample = [bob_sifted[i] for i in sample_indices]

    errors = sum(a != b for a, b in zip(alice_sample, bob_sample))
    error_rate = errors / sample_size

    # Remaining bits form the key
    remaining_alice = [bit for i, bit in enumerate(alice_sifted) if i not in sample_indices]
    remaining_bob = [bit for i, bit in enumerate(bob_sifted) if i not in sample_indices]

    return error_rate, remaining_alice, remaining_bob


In [85]:
#Step 7: Optional — Simulate Eavesdropper (Eve)
#Eve measures qubits randomly and re-encodes them, introducing errors.
def eavesdrop(circuits, eve_bases):
    backend = Aer.get_backend('qasm_simulator')
    intercepted = []
    for qc, eve_basis in zip(circuits, eve_bases):
        q = qc.copy()
        if eve_basis == 1:
            q.h(0)
        q.measure_all()
        job = execute(q, backend, shots=10)
        result = job.result()
        eve_bit = int(list(result.get_counts().keys())[0])

        # Re-encode (may disturb state)
        new_qc = QuantumCircuit(1, 1)
        if eve_bit == 1:
            new_qc.x(0)
        if eve_basis == 1:
            new_qc.h(0)
        intercepted.append(new_qc)
    return intercepted


In [86]:
#Step 8: Combine Everything
def bb84_step_by_step(n=20, eavesdropper=False):
    print("=== BB84 Quantum Key Distribution Simulation ===")
    print(f"Total qubits: {n}\n")

    # Step 1: Alice generates random bits and bases
    alice_bits = generate_random_bits(n)
    alice_bases = generate_random_bases(n)
    print("Alice's bits: ", alice_bits)
    print("Alice's bases:", alice_bases)

    # Step 2: Alice encodes qubits
    circuits = create_bb84_qubits(alice_bits, alice_bases)

    # Step 3: Optional eavesdropper (Eve)
    if eavesdropper:
        eve_bases = generate_random_bases(n)
        circuits = eavesdrop(circuits, eve_bases)
        print("\n⚠️  Eve is eavesdropping!")

    # Step 4: Bob chooses bases and measures
    bob_bases = generate_random_bases(n)
    bob_bits = measure_qubits(circuits, bob_bases)
    print("\nBob's bases:       ", bob_bases)
    print("Bob's measurements:", bob_bits)

    # Step 5: Alice & Bob sift their keys
    alice_sifted, bob_sifted = sift_key(alice_bits, alice_bases, bob_bits, bob_bases)
    print(f"\nMatching bases: {len(alice_sifted)} / {n}")
    print("Sifted (Alice):", alice_sifted)
    print("Sifted (Bob):  ", bob_sifted)

    # Step 6: Publicly check subset for eavesdropping
    error_rate, final_alice, final_bob = check_for_eavesdropping(alice_sifted, bob_sifted)
    print(f"\nError rate from sample: {error_rate:.2%}")
    if error_rate > 0.25:
        print("🚨 High error rate! Eavesdropper detected — key discarded.")
    else:
        print("✅ Secure channel. Shared key established.")

    print("\nFinal secret key:")
    print("Alice:", final_alice,"len_key=",len(final_alice))
    print("Bob:  ", final_bob,"len_key=",len(final_bob))

# --- Run ---
bb84_step_by_step(n=no_qubits, eavesdropper=False)


=== BB84 Quantum Key Distribution Simulation ===
Total qubits: 200

Alice's bits:  [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1]
Alice's bases: [0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 

In [89]:
# Encrypt using BB84 key (one-time pad)
plaintext = "HELLO"
key = [1, 0, 1, 1, 0]  # from BB84 for example
binary_text = ''.join(format(ord(c), '08b') for c in plaintext)
cipher = ''.join(str(int(b) ^ key[i % len(key)]) for i, b in enumerate(binary_text))
print(cipher)

1111110111101000001001110001011010011001
